# Phylogeny from whole genome sequence data

When we sequence a population, we aim to capture the variation (SNPs, indels, gene gain and loss etc.) in the samples and use it to infer the relationships between the samples. Two of the main approaches to capturing this variation and reconstructing the bacterial genomes are:

* De novo genome assembly and annotation
* Mapping and variant calling against a reference genome

Each approach has it's benefits and limitations. We will focus on mapping and variant calling in this tutorial. For mapping and variant calling, whether we are dealing with different bacterial isolates, with viral populations in a patient, or even with genomes of different human individuals, the principles are essentially the same. Instead of assembling the newly generated sequence reads de novo to produce a new genome sequence, it is easier and much faster to align or map the sequence reads to a reference genome. We can then readily identify SNPs and indels that distinguish closely related populations or individual organisms and may thus learn about genetic differences that may cause drug resistance or increased virulence in pathogens, or changed susceptibility to disease in humans. One important prerequisite for the mapping of sequence data to work is that the reference and the re-sequenced subject have the same genome architecture.

In this exercise, we will use sequence data from _Salmonella enterica serovar Typhi_ samples to demonstrate the mapping and variant calling approach. Importantly, although the data is based on real sequence data, it has been edited to make it run more efficiently for the purpose of this tutorial.

Navigate to the directory that contains the sequence data:

In [ ]:
cd ~/course_data/snp-phylogeny/data/typhi

Take a look at the directory containing the sequence data for the samples:

In [ ]:
ls fastq

## Introducing the tutorial dataset

We will use data adapted from the following paper:

> **A genomic snapshot of Salmonella enterica serovar Typhi in Colombia**  
> Guevara, Paula Diaz, et al.  
> _PLoS Neglected Tropical Diseases2021. doi: 10.1371/journal.pntd.0009755_  
> PMID: [34529660](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8478212/) 

_Salmonella enterica serovar Typhi_ (_S. Typhi_) is the causative agent of typhoid fever, with between 9–13 million cases and 116,800 associated deaths annually. Typhoid fever is still a public health problem in many countries, including in Latin America, which has a modelled incidence of up to 169 (32–642) cases per 100,000 person-years. Several international studies have aimed to fill data gaps regarding the global distribution and genetic landscape of typhoid; however, in spite of these efforts Latin America is still underrepresented. This study provided the first enhanced insights into the molecular epidemiology of S. Typhi in Colombia, using whole genome sequencing data to investigate the population structure in Colombia and identify predominant circulating genotypes.

## Overview of mapping and variant calling approach

The diagram below illustrates the steps involved when mapping and calling variants for a set of bacterial samples.

![Approach](img/snp-phylogeny-approach.png)

The first step once you have obtained your sequence data (FASTQ) is to QC the data. After QC, the sequence data is matched or aligned to a reference genome (FASTA) in a process called read mapping to produce a set of read aligments (SAM/BAM). These read alignments are inspected to identify differences between the aligned reads and the reference genome. This process is called variant calling and produces VCF files. In fact during this process we capture information about every position in the genome (variant and non-variant sites) in the VCFs. Each site in the VCF has a set of quality filters applied and any sites identified as low quaility (e.g. less than 4 reads aligned at that position) are marked as low quality in the VCF to produce a filtered VCF. We use this filtered VCF file in a process called consensus caling to reconstruct a consensus _pseudosequence_ or _pseudogenome_ for our sample (FASTA). In the _pseudogenome_, any sites marked as low quality will be represented as an N in the reconstructed sequence. These pseudogenomes (multi-FASTA) are then aligned and variation identified and used to reconstruct a phylogeny of our samples. 

## Exercise

Now let's analyse some data!

### Prepare the data

First take a look at the sequence data provided.

In [ ]:
ls fastq/

#### Check your understanding

1. How many samples have been sequenced?
2. How many fastq files are there? 


We will use the chromosome sequence of _Salmonella typhi CT18_ as the reference genome. This has already been downloaded from RefSeq. Take a look at the reference genome:

In [ ]:
ls ref/

Check the size of the reference file:

In [ ]:
assembly-stats ref/Styphi_CT18.fa

Now use bwa to index the reference genome. This creates a lookup table that bwa uses when matching the sequence reads against the reference genome.

In [ ]:
bwa index ref/Styphi_CT18.fa

#### Check your understanding

3. How many sequences in the reference fasta file?
4. What are the names of the sequences in the reference fasta file?
5. What is the size of the reference? 
6. What additional files did the indexing step produce?

### QC the sequence data

An important first step in any analysis is QC of the data. We will the FastQC software to QC the data. First create a directory for the qc results:

In [ ]:
cd fastq
mkdir qc_results

Run FastQC on the all the fastq files and store the results in the directory `qc_results`:

In [ ]:
fastqc -o qc_results *.fastq.gz

This will create one html report for each of the fastq files. Take a look:

In [ ]:
ls qc_results/

If we have lots of samples it will be difficult to manually inspect each file. Therefore we will use multiQC to collate all the QC reports into one file.

In [ ]:
multiqc qc_results

Open the collated report `multiqc_report.html` in firefox.

In [ ]:
firefox multiqc_report.html &

![MultiQC results](img/multiqc.png)

#### Check your understanding

1. What is the median read length for sample ERR5243693?
2. Which sample has the largets yield (mose sequence data)?

### Trim the reads to remove low quality and adapter sequence

If your sequence reads have a high level of adapter contamination and/or have low quality bases at the end of the reads you can trim the reads to remove these sequences. There are several software that can be used to do this including trimmomatic and fastp.

Use fastp to trim the reads for sample ERR5243693.

In [ ]:
fastp \
   --in1 ERR5243693_1.fastq.gz --in2 ERR5243693_2.fastq.gz \
   --out1 ERR5243693_1.trim.fastq.gz \
   --out2 ERR5243693_2.trim.fastq.gz \
   --json ERR5243693.fastp.json --html ERR5243693.fastp.html \
   --detect_adapter_for_pe --cut_mean_quality 20 \
   --thread 2

Now repeat for the other samples.

Take a look at the output from fastp:

In [ ]:
head -20 ERR5243693.fastp.json
head -20 ERR5243695.fastp.json
head -20 ERR5243699.fastp.json

#### Check your understanding
1. How much data (bp/base pairs) was lost due to trimming and adapter removal?

### Map the data to a reference genome 

When performing any data analysis it is good practice to arrange your data in a logical way rather than putting all files in a single directory. Let's create a directory to store the results of mapping the reads to the reference genome:

In [ ]:
mkdir ../samtools
cd ../samtools

Use `bwa` to map the reads for sample ERR5243693 to the reference genome.  

In [ ]:
bwa mem -t 2 ../ref/Styphi_CT18.fa \
../fastq/ERR5243693_1.trim.fastq.gz ../fastq/ERR5243693_2.trim.fastq.gz > \
ERR5243693.sam

This may take a few minutes to run. Let's take a look at the options:

* `-t` :  tells the program to use 2 CPUs
* `../ref/Styphi_CT18.fa` is the reference to match the reads against
* `../fastq/ERR5243693_1.trim.fastq.gz and ../fastq/ERR5243693_2.trim.fastq.gz` are the fastq files containing our sequence reads (after trimming) for sample ERR5243693
* the output is SAM format and `> ERR5243693.sam` redirects the output to the file `ERR5243693.sam`

When complete, convert the sam file to a bam file with samtools:

In [ ]:
samtools view -@ 2 -bh -o ERR5243693.bam ERR5243693.sam

The `-@` option tells the program to use 2 CPUs and the `-b` option specifies to write the output as a bam file, `-h` option means include the header information in the output. The `-o` option specifies the name of the output file `ERR5243693.bam`.

Sort the bam file and then index the sorted bam file:

In [ ]:
samtools sort -@ 1 -o ERR5243693.sorted.bam -T ERR5243693.sorted ERR5243693.bam

In [ ]:
samtools index ERR5243693.sorted.bam

Generate some statistics about the alignment:

In [ ]:
samtools stats ERR5243693.sorted.bam > ERR5243693.stats
samtools flagstat ERR5243693.sorted.bam > ERR5243693.flagstat
samtools coverage ERR5243693.sorted.bam > ERR5243693.coverage

Now repeat for the other samples.

#### Check your understanding
1. What %reads mapped to the reference for each sample? 
2. What %genome was covered for each sample? 
3. What is the mean depth of coverage for each sample? 

### Call variants

Go through each position in the reference genome and look at reads aligned at that position and make a call about what the base is at that position for the sample. This information will be stored in a VCF file and if there are any differences then this will be marked as a variant (snp/indel) in this VCF file. 

Again create a directory to store the results of the variant calling step:

In [ ]:
mkdir ../variants
cd ../variants

Run this for sample ERR5243693 using bcftools:

In [ ]:
bcftools mpileup --fasta-ref ../ref/Styphi_CT18.fa \
--min-BQ 20 \
--annotate \
FORMAT/AD,FORMAT/ADF,FORMAT/ADR,FORMAT/DP,FORMAT/SP,INFO/AD,\
INFO/ADF,INFO/ADR ../samtools/ERR5243693.sorted.bam | bcftools call \
--output-type v --ploidy 1 --multiallelic-caller - | \
bcftools view --output-file ERR5243693.vcf.gz --output-type z

Again this may take some time to run. Let's look at the options:

The `bcftools mpileup` command is passed the following options:

* `--fasta-ref ../ref/Styphi_CT18.fa` is the reference that the reads were mapped to
* `--min-BQ 20` 
* `--annotate FORMAT/AD,FORMAT/ADF,FORMAT/ADR,FORMAT/DP,FORMAT/SP,INFO/AD,INFO/ADF,INFO/ADR` tells the program what tags to incude in the VCF

The VCF produced by the bcftools mpileup command is passed to `bcftools call` with the following options:

* `--output-type v` tells the program to output VCF format
* `--ploidy 1` tells the program that we are daling with haploid datasets
* `--multiallelic-caller` tells the program which variant calling algorithm to use

The VCF produced by the `bcftools call` command is passed to `bcftools view` with the following options which converts the VCF to a compressed VCF file:

* `--output-file ERR5243693.vcf.gz` tells the program the name of the output file
* `--output-type z` tells the program to create a gzipped VCF file as output

When it completes, index the gzipped VCF file that has been created:

In [ ]:
tabix -p vcf -f ERR5243693.vcf.gz

Take a look at the VCF file that was produced:

In [ ]:
bcftools view ERR5243693.vcf.gz | less -S

Notice that the VCF has information about every site in the genome.

Now generate some statistics about the VCF file:

In [ ]:
bcftools stats ERR5243693.vcf.gz > ERR5243693.vcf.stats.txt

Now repeat for the other samples.

Look at the statistics for the variant calling:

In [ ]:
less ERR5243693.vcf.stats.txt
less ERR5243695.vcf.stats.txt
less ERR5243699.vcf.stats.txt

#### Check your understanding
1. How many sites are in the VCF file for each sample?
2. Does this match to the size of the reference used in the read mappping step?
3. How many variant sites were identified for each sample?

### Filter the VCFs
We want to identify calls where we have a high confidence that they are correct (and not due to sequencing errors and/or misalignment of the reads). We use criteria like read depth at a position, quality scores etc. to filter out low quality calls at each position.

Use bcftools to filter sites for sample ERR5243693.

In [ ]:
bcftools filter \
--output ERR5243693.filtered.vcf.gz \
--soft-filter LowQual \
--exclude "QUAL<25 || FORMAT/DP<10 || MAX(FORMAT/ADF)<2 || MAX(FORMAT/ADR)<2 \
|| MAX(FORMAT/AD)/SUM(FORMAT/DP)<0.9 || MQ<30 || MQ0F>0.1" \
--output-type z ERR5243693.vcf.gz

The filtering step may take some time to run. Let's look at the options used in the command above:

* `--output` specifies the name of the output file
* `--soft-filter` tells the program to keep any filtered position in the file and mark them as `LowQual` rather than remove them completely from the file (hard filter)
* `--exclude` lists the filtering criteria to apply to each position
* `--output-type` specifies the type of output file to create, in this case z means a xompressed VCF file

When the bcftools filter command is complete, index the filtered VCF file:

In [ ]:
tabix -p vcf -f ERR5243693.filtered.vcf.gz

Take a look at the VCF file and notice how some of the sites are marked as `PASS` or `LowQual` under the filter column.

In [ ]:
bcftools view ERR5243693.filtered.vcf.gz | less -S

#### Check your understanding

Looking at the filtered VCF for sample ERR5243693:

1. Does position 2 pass or fail?  
2. Does position 244 pass or fail?  
3. What is the reason for the failure at position 2527?

To view only the sites that passed the filtering step:

In [ ]:
bcftools view -f PASS ERR5243693.filtered.vcf.gz | less

To generate some statistics about the filtered VCF file:

In [ ]:
bcftools stats ERR5243693.filtered.vcf.gz > \
ERR5243693.filtered.stats.txt

Now repeat for the other samples.

#### Check your understanding
4. How many sites were marked as low quality in the filtering step?
5. How many variant sites were marked as low quality in the filtering step?

### Call a consensus sequence for each sample

A pseudogenome is a reconstruction of what we think the genome is for the sample using the reference genome as a basis. To create it for a sample, you go through each position in the reference and determine what base is called (using the VCF from the previous steps) for the sample. Sometimes this will be the same as the reference, and sometimes it will differ from the reference (a variant). For positions that are flagged as low quality/filtered out (e.g. no reads covering the position) we use an N in the pseudogenome. This is because you cannot be confident what the base is at this position for the sample. In the end the length of the pseudogenome for your sample should be the same as the length of the reference. 

To create a pseudogenome for sample ERR5243693 use the script _vcf2pseudogenome.pl_. This has already been installed on the computer using this command (it also has a dependency on _pysam_ and _biopython_):

`wget https://raw.githubusercontent.com/nf-core/bactmap/master/bin/vcf2pseudogenome.py`

Create a directory to store the pseudogenomes:

In [ ]:
mkdir ../pseudogenomes
cd ../pseudogenomes

The run the `vcf2pseudogenome.py` script:

In [ ]:
vcf2pseudogenome.py -r ../ref/Styphi_CT18.fa \
-b ../variants/ERR5243693.filtered.vcf.gz -o ERR5243693.fas

Now repeat for the other samples.

#### Check your understanding
1. What is the length of the pseudogenomes? (Hint: Use assembly-stats)
2. Does it match the length of the reference?

### Create a multiple sequence alignment of all pseudogenomes
Remember that to reconstruct the phylogeny of our samples we need a multifasta alignment of our sequences. We are going to use a reconstruction of the entire genome as our set of sequences.

In [ ]:
cat *.fas > aligned_pseudogenomes.aln

Because of the way the pseudogenomes were constructed resulting in them all being the same length we do not have to perform a multi-sequence alignment to ensure all the bases are homologous.

#### Check your understanding
1. How many sequences in the multiple sequence alignment file of pseudogenomes?
2. What is the largest and mean sequence length? (Hint: Use assembly-stats)

### Add the reference genome to the multiple sequence alignment
Sometimes you may want to include the reference genome in your tree. To do this, add the reference sequence to the multifasta alignment of your samples.

In [ ]:
cat ../ref/Styphi_CT18.fa aligned_pseudogenomes.aln \
> ref_and_aligned_pseudogenomes.aln

At this point it would be useful to look at your alignment in a multiple sequence alignment viewer e.g. seaview.

In [ ]:
seaview ref_and_aligned_pseudogenomes.aln &

#### Check your understanding
1. How many sequences in the multiple sequence alignment file containng the reference genome and sample pseudogenomes?

### Mask repetitive regions (optional but good practice)
Bases called in repetitive regions may not be true variation (e.g. due to misalignment of reads) and may compromise the core phylogenetic signal. Therefore it is good practice to identify known repetitive regions and mask these out from your alignment.

To achieve this, either a file of known regions for the reference you mapped to will exist (see the literature) or one can be generated by matching the reference genome against itself (to identify repeat regions) with e.g. Mummer and Phast used to identify prophage. You can use the software remove_blocks to mask out any repetitive region in your multifasta alignment, this masking involves replacing the bases with Ns in this region. To save time we wont run this step here.

### Draw a tree with iqtree
Now that we have a multiple sequence alignment, we can use IQ-TREE to build a maximum likelihood phylogeny. But first create a directory to store the iqtree analysis.

In [ ]:
mkdir ../iqtree
cd ../iqtree

Calculating a phylogeny on whole genome sequences can be very time consuming. We can speed this up by only using the variable sites (SNPs). These are sites that differ in at least one of the samples. However, we need to be aware that only including variable sites can affect the evolutionary rate estimates made by phylogenetics software - therefore, we need to account for the sites we remove in our analysis.

We will use snp-sites to do this. You can view the options for snp-sites:

In [ ]:
snp-sites -h

First, remove all the invariant sites and create a SNP-only multiple sequence alignment:

In [ ]:
snp-sites -o pseudogenomes.snps.aln \
../pseudogenomes/ref_and_aligned_pseudogenomes.aln

Look at the new file that is created:

In [ ]:
less pseudogenomes.snps.aln

We can also see how many invariant sites were removed (and what proportion of A, T, G, C they were) using

In [ ]:
snp-sites -C ../pseudogenomes/ref_and_aligned_pseudogenomes.aln

#### Check your understanding

1. How many variant sites are identified?
2. How many invariant sites are identified?
3. Does this correlate with the expected number i.e. from the literature?

Now look at the options for IQ-TREE:

In [ ]:
iqtree -h

And then construct the tree with IQ-TREE:

In [ ]:
iqtree \
    -s pseudogenomes.snps.aln \
    -fconst $( snp-sites -C ../pseudogenomes/ref_and_aligned_pseudogenomes.aln ) \
    -m GTR+G+T \
    -B 1000 \
    -T 2 \
    -mem 2GB

In the command above, we have used te following options

* `-s` to specify the multiple sequence alignment file `pseudogenomes.snpsites.aln`
* `-fconst` to ask IQ-TREE to take account of missing invariant sites $(snp-sites -C ref_and_aligned_pseudogenomes.aln calculates the specific values to pass to iqtree)
* `-m` to specify an evolutionary model, we want IQ-TREE to use -m GTR
* `-B` to perform 1000 ultrafast bootstraps 
* `-T` to tell IQ-TREE to use a maximum of 2 CPUs (threads) 
* `-mem` 2GB to tell IQ-TREE to use a maximum of of 2GB memory

Our maximum likelihood tree is labelled `pseudogenomes.snpsites.aln.treefile`. The treefile suffix is not always correctly identified by many tools, so we'll relabel this as something else:

In [ ]:
cp pseudogenomes.snps.aln.treefile pseudogenomes.snps.aln.tre

We can look at the raw text file:

In [ ]:
cat pseudogenomes.snps.aln.tre

But it is better if we visualise this using a tree view e.g.figtree

In [ ]:
figtree pseudogenomes.snps.aln.tre &

### Accounting for recombination with Gubbins

Variation due to recombination events can mask the core phylogenetic signal, therefore it is recommended to identify these regions in your alignment and mask them out. We can use gubbins to infer recombining sites by looking for increased SNP density that occurs in specific ancestral nodes. Look at the options for gubbins:

In [ ]:
run_gubbins.py -h

Make a directory to store the results of running gubbins:

In [ ]:
mkdir ../gubbins
cd ../gubbins

Run gubbins on your data:

In [ ]:
run_gubbins.py -c 2 -p gubbins \
../pseudogenomes/ref_and_aligned_pseudogenomes.aln

The `-c` option tells the program to use 2 CPUs and the `-p` option tells the program to name all files with the prefix gubbins. This command can take a few minutes to run.

__Note:__ Gubbins must be run on the entire alignment (not just SNPs) as it uses the spatial distribution to identify and filter out recombinant regions.

NB. If gubbins takes more than 10 mins to complete, we have already run it for you - the files are available at:

In [ ]:
ls -l ~/course_data/snp-phylogeny/data/typhi/gubbins_backup

Lets look at what gubbins has done:

In [ ]:
ls -l *

You can explore these files. For example `gubbins.recombination_predictions.gff` is a gff file that contains a record of each recombination block identified, how many SNPs it contains, and what samples are affected.

head gubbins.recombination_predictions.gff

In [ ]:
The file `gubbins.final_tree.tre` is a phylogeny in which recombination has already been accounted for.

You can visualise this in figtree.

In [ ]:
figtree gubbins.final_tree.tre

## Root the phylogeny

The trees generated by iqtree and gubbins are unrooted, but we may want to apply some evolutionary direction to them. One strategy for rooting a tree is called _midpoint rooting_. Midpoint rooting involves locating the midpoint of the longest path between any two tips and putting the root in that location. Note that this does not necessarily infer the true root, and this should be used with caution.

To midpoint root our tree, we will use a simple script written in python that uses the ete package. You can examine the code:

In [ ]:
less ../midpoint.root.py

![Python script to midpoint root a tree](img/midpoint.root.image.png)

Run this script to midpoint root the tree.

In [ ]:
python midpoint.root.py gubbins.final_tree.tre > gubbins.final_tree.midpoint.tre

Visualise this in figtree.

Another common strategy for rooting the tree is _outgroup rooting_. This is the preferred approach for bacterial datasets. Outgroup rooting involves including one or more sequences in the analysis that are more distantly related to our sequences of interest than they are to one another. These sequence are usually referred to as _outgroups_. The root estimate is then simply the point at which the outgroup(s) join the tree. The best possible outgroups are those available which are most closely related to our sequences of interest but still different enough. For examples, in this dataset we could use a _Salmonella paratyhi A sample_ as an outgroup.

There are a few ways to do this. One is to obtain sequence data for the outgroup sample and incorporate it into the dataset from the beginning of the analysis and construct a pseudogenome for the outgroup. If no sequence data is available, you can take a complete reference genome for the outgroup and `shred` it to simulate sequencing reads for the reference. Just remember when calculating and reporting the number of variant sites for your dataset that you remove the outgroup from the alignment.

### Clean-up!

Clean up any intermediate files that were generated during the analysis that you no longer require. This is always an important last step of any analysis as sequence data analysis files can use up large amounts of disk space. Let's look at what files were created in our analysis

In [ ]:
cd ..
ls -alhrt */*

As you can see the data analysis has generated a lot of files! So let's remove any files that do not need to be kept long term:

In [ ]:
rm fastq/*.trim.fastq.gz
rm fastq/*.fastp.html
rm fastq/*.fastp.json
rm samtools/*.sam
rm samtool/ERR5243693.bam*
rm samtools/ERR5243695.bam*
rm samtools/ERR5243699.bam*
rm variants/ERR5243693.vcf.gz*
rm variant/ERR5243695.vcf.gz*
rm variants/ERR5243699.vcf.gz*

Now go to the next section: [Phylogeny and Metadata](metadata.ipynb)